In [ ]:
import thinkplot
import thinkstats2
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import scipy.stats as ss
import math
import random

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

# Multiple Regression

Doing a simple linear regression is cool, but for this to be really useful we want to be able to use a bunch of factors to make a prediction. The process is largely the same, we just have more variables on the input (X) side. It is also difficult/impossible to visualize, since we'd need to see in 3+ dimensions.  

## Exploratory Data Analysis

First, we'll get some data, and look at what the target and the features are going to be. This is a really simplifed example of some EDA that we normally do in advance of machine learning. The first step to any predictive model building is to look at our data and see if anything should be done prior to modelling. This is pretty vague, but we are basically looking to see if there's anything in the data that might be suboptimal for the predictions we want to create. Some common things to look for are:
<ul>
<li> Errors - anything that is just "wrong", words in numeric columns, mislabeled values, etc...
<li> Outliers - values too large or too small. 
<li> Weird Distributions - this is very open-ended, but think of something like a numerical feature where the variance is exceeding low. If every row of the data has the same value, it probably won't help in making predictions, which is inheirently seeking to discriminate. For example, some schools in Japan used to force students to dye their hair black if it wasn't already; if hair color was a feature in your dataset, it wouldn't really add any value if 99.9% of rows of data is "black". 
<li> Missing Values - how should we deal with it if some values are blank?
</ul>

This isn't a conclusive list, and as we explore we may find other things to change or remove. The amount of work we may need to do can also vary widely - if the data is comming from something like a PoS system it will probably be pretty clean and not require many changes; if the data is coming from scraping social media sites from that people fill out, the data may well require massive amounts of cleaning. 

In [ ]:
#Get data
#I'll drop density to make it more realistic.
df = pd.read_csv("data/bodyfat.csv")
df = df.drop(columns={"Density"})
df.head()

Cool. We'll predict the BodyFat (which is hard to accurately measure) by using the other stuff, which is easier to measure. 

Before we start, we want to do some exploration of the data.

In [ ]:
#Start Exploring the Data
df.describe()

At a glance, it looks like there aren't a lot of issues. One thing I notice is that there's a BodyFat reading of 0 for the min. That isn't possible, so we'll need to get rid of that. 

In [ ]:
#Get variable info. 
df.info()

All of our variables are correctly identified as numbers. Often if there's some erroneous data in a column it might get miscategorized as an object. Since we'll be dealing with non-numeric data in ML, starting next workbook, we probably want to double check.  

In [ ]:
#Visual correlation and distributions
sns.pairplot(df)

It appears there are some outliers that probably aren't helping - e.g. Height. 

##### Check for Missing Values

In [ ]:
#Are there nulls?
df.isnull().sum().sort_values()

##### Filter Outliers

In [ ]:
# Remove some outliers
# These were judgement cals. 
# E.g. it is theoretically possible for someone to have fat < 5%, but that's really only people like
# bodybuilders at a competition time. Those types of extreme outliers probably wont' help our model. 
df = df[df["BodyFat"] > 5]
df = df[df["Height"] > 40]
df = df[df["Weight"] < 300]
df = df[df["Ankle"] < 30]
df.describe()

##### Examine Distributions Post-Outliers

In [ ]:
#Check with cleaned data. 
sns.pairplot(df)

Looks pretty good. We may want to examine some of the outliers a bit more closely, but overall the data looks pretty usable. Distributions are all pretty normal. Large outliers are gone. Nothing really jumps out as being a problem.

There does appear to be a large amount of correlation between variables. We'll worry about that later on, for now, looks good. Time to start regressing...


## Setup the data

The basic idea here is the same as one variable regression, only instead of one X, we have a bunch. We can't do this with simple functions (thinkstats), at least not easily, so we'll use a package. 

### SciKitLearn Data Setup

In [ ]:
#Y data is simple, do that first
y = np.array(df["BodyFat"]).reshape(-1,1)
y.shape

Y is identical to the single variable stuff. For X, we have a width to our array. We can take a look again to see what we expect. 

In [ ]:
#Get a new df with only the features we'll use
df_ = df.drop(columns={"BodyFat"})
df_.head()

In [ ]:
#Make that df into an array. 
x = np.array(df_)
x.shape

In [ ]:
#shapes
print("X shape", x.shape)
print("Y shape", y.shape)

### SKL Regress...

Shape for this one looks good! We have 240 rows, which matches the Y. We have 13 columns, which is what we get if we were to count up the columns by hand. Success!!

Time for regression stuff. 

In [ ]:
#Setup
from sklearn.linear_model import LinearRegression
from sklearn import feature_selection
from sklearn.model_selection import train_test_split

#### Split Data

In [ ]:
#Split data
xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size=.3)

##### Train Model

In [ ]:
#Generate model 
model = LinearRegression().fit(xTrain,yTrain)

##### Check Results

In [ ]:
#Get some info on our new regression model
r_sq = model.score(xTest, yTest)
print('R-squared:', r_sq)

#### View "Slopes" and Intercept

In single regression our model was made up of the slope, and intercept values - we can plug any X in and calculate our prediction. In multiple regression, the results are the same, there's just more of them. We still have one intercept, but now each X has it's own m. The entire equation ends up just having a bunch of "X" terms:

$ y = m1 * x1 + m2 * x2 + m3 * x3 ... + b $

We can extract the values just as we did before, we could also make a manual model to generate predictions - our equation would just be longer than the y = mx + b ones that we created earlier. 

#### Visualizing Multiple Regression

The idea of how multiple regression works is the same, it is just harder to visualize. If we try to look at an example with 2 features (so 3 total values, with the target) and picture it a little. In a single regression we make a prediction by doing a "lookup" in one dimension - the X value is our input, we slide along the X axis until we find that value, then look up to the regression line to see what the target is at that point. In a 2 feature regression, the same idea applies, only we do the lookup with two X values on a plane and the prediction is where they intersect on the 3rd axis:

![3D Regression](images/3d_regression.png "3D Regression")

Now our model, or our line of best fit, is a plane - a two dimensional line. Each feature we add ups that number of dimensions by 1, and it is always 1 less than the total number of dimensions. In a single regression teh value on one axis predicts the other axis, in a multiple regression the value on [# of features] axis, where they interesect, predicts the value on the other axis. Same, same. 

Going up in features, the same idea always applies, we just don't have a timecube to draw it. If we had 13 features like we do above, we'd have a scatter plot in 14 dimensional space, and we'd look to the intercept of the 13 features, which would give us our prediction on the 14th axis - the one belonging to the target. 

In [ ]:
#Our coefficent/slope is now an array of values - one per X. 
#Visualizing the regression would be a 14D space, where these are the slopes in each dimension. 
print('Intercept:', model.intercept_[0])
print('Coefs:', model.coef_[0])

##### Error Metrics

Get the residuals to calculate RMSE. 

The residual stuff is the same no matter how many Xs we have on the input side, since all we are checking is the Y values. 

In [ ]:
#Get RMSE
tmp = model.predict(xTest)
mean_squared_error(tmp, yTest, squared=False)

##### Residuals

We don't generally need to view the residuals if we aren't doing things by hand, the library functions allow us to just look at the summary statistics like RMSE. We can view them for fun, just to confirm they exist. 

In [ ]:
#Get Residuals and picture them in a DF for easy reading. 
tmp1 = pd.DataFrame(yTest, columns={"Y values"})
tmp2 = pd.DataFrame(tmp, columns={"Predictions"})
tmp3 = pd.DataFrame((yTest-tmp), columns={"Residual"})
resFrame = pd.concat([tmp1,tmp2,tmp3], axis=1)
resFrame.head()

Easy!!

<h3>Statsmodels style</h3>

In [ ]:
#Statsmodels.
import statsmodels.api as sm

In [ ]:
#Fit the model. 
X2 = sm.add_constant(xTrain)
est = sm.OLS(yTrain, X2)
est2 = est.fit()
print(est2.summary())

The x's are the input features in order. They are showing without their names because for the input we just used the arrays from the sklearn example above. Things like this are reasonably common when we're doing ML, as most other algorithms don't really give detailed data for each feature like linear regression does, so having them listed by names isn't super useful. To get the labels we could:
<ul>
<li>Feed the statsmodel a formula, like we have in the book. 
<li>Use a dataframe as an input. (Coming up)
<li>Create a list of column names and match the up after the fact - reconstructing output that you want from the values you want.
</ul>

In [ ]:
#RMSE
ypred = est2.predict(sm.add_constant(xTest))
mean_squared_error(yTest,ypred, squared=False)

## Exercise

In [149]:
# Load data and remove bodyfat
dfE = pd.read_csv("data/bodyfat.csv")
dfE.drop(columns={"BodyFat"}, inplace=True)
dfE.head()

,Density,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,1.0853,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,1.0414,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,1.0751,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,1.0340,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [155]:
#Setup Data
yE = np.array(dfE["Density"]).reshape(-1,1)
xE = np.array(dfE.drop(columns={"Density"}))
xE.shape, yE.shape

((252, 13), (252, 1))

In [156]:
#Split and Train
xTrainE, xTestE, yTrainE, yTestE = train_test_split(xE,yE,test_size=.3)
#Generate model 
modelE = LinearRegression().fit(xTrainE,yTrainE)

In [157]:
#Score 
#Get RMSE and R2
tmpE = modelE.predict(xTestE)
rmseE = mean_squared_error(tmpE, yTestE, squared=False)
r_sqE = modelE.score(xTestE, yTestE)

print("RMSE:", rmseE)
print('R-squared:', r_sqE)

RMSE: 0.011128385792870505
R-squared: 0.6505217281635629


## Multicollinearity - Useful, but Less Critical

For fun, we'll look at the multicollinearity, because it is crazy high in this example. Look back at the pairplot, most varaibles appear highly correlated with each other - as one increases, the other does as well. This makes sense logically, as these are all measures of the body size; as your wrist size increases, so does your forearm, and your bicep, etc... They are closely correlated. One effect of this is that the impact of each variable is hard to pinpoint, as all the varaibles are largely measuring the same thing. 

Addressing this won't make our accuracy measures get way better, but it will allow us to better attribute the impact to the individual features, which is one advantage to a linear regression - that table of results give us some data that we can use to edit our model. 

We can calculate the VIF - variance inflation factor. This is a measure of multicollinearity. The calculation is VIF = 1/(1-R2), so as R2 gets closer to 1, the VIF gets larger. If we think of it in R2 terms - a big R2 indicates that a large percentage in the varaince of the results is captured in the model. Here, the R2 is how much of the variance of the other features is explained by each feature. So if they are higly correlated that value will be high and R2 will be high, leading to a big VIF value. If the R2 is low, then the varaible doesn't capture varaiance in the other features, so it is different from them, or it captures different data. In this case the R2 is small, so the VIF is small. 

The rule of thumb is that a VIF of ~10 should be looked at. 

### Variance Varies

One way that makes a lot of sense to me to think of in this case, R2, and a few other places is to think of a measure of how much the data varies, and what is "causing" that varaince - as an intuitive sense, not a literal causitive relationship. 

Here, we can start with the assumption that the target varries - it literally just takes on different values. 
<ul>
<li> If we look back to R2, that's a measure of how much of that variance is captured in the model - or put in this context, the target varries a bunch, and R2% of that variation is "caused" (or captured by, more accurately) the things that we have in our model, and (1-R2%) is due to other stuff - whatever our model is missing. A complete model captures most of the things that cause the data to differ - if we step back and think about it logically and not statistically, this makes a lot of sense to me personally. 
<li> If we look at ANOVA (the f-score in particular) what we are looking for is a measure of what varies more. Do we see more change (variance in the data) when comparing two groups, which is an indication that the groups are different, or do we see more change when comparing the values inside of a group (indicating the groups are more similar). 
<li> Here, we are ultimately looking to attribute the varaiance in the target to the set of features that really matters. When measuring the VIF, we are basically asking the question, "does the data still vary just as much without this varaible?" If we have a high VIF that tells us that this varaible doesn't really "add any information" - the data varies almost/just as much with it gone, so it wasn't a determinative factor in making the data change. If the VIF is low, that tells us that the amount the data changes (varies) is dependent on this varaible - it contains information that makes the data change.
</ul>

Again, this is a mental model that makes sense to me, it is not a specific calculation or rule. On the whole, in predicive analytics, we are attempting to determine "what causes this data to vary", so we can use those things to make a prediction. The more of that varying we capture, the better we generally do. We can also look backwards, like we are here, and attempt to attribute that varaiance we see to different "causes", so we can do stuff like remove the ones that aren't making that much of an impact. 

In [ ]:
# Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Function will check the VIF of each variable in a DF and return the results in another DF
def calc_vif(X):
    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

In [ ]:
#Check VIFs
calc_vif(df_)

##### View the Correlations

In [ ]:
sns.pairplot(df[["Hip", "Knee", "Wrist", "Chest","Neck","Thigh","Ankle"]])

#### Filter Out the Values

All the scores are huge. We expected this since they all almost overlap on that pairplot earlier. What this tells us is that the information in each varaible has lots of overlap - in this case they all measure bodyfat, fairly directly. If we think about the real world impact of this it makese sense - the cheap and easy way to measure bodfat is by taking measurements and caliper pinches all over one's body. We'd expect the measurements to all be pretty correlated - it'd be a bit odd to have a huge wrist and a tiny ankle, or a big thigh and tiny knee. A few people might stand out, particularly very muscular people, but most people will be pretty consistent between every measurement based on fat level. 

How accurate will this model be if we drop a bunch of the high VIF stuff?

In [ ]:
#Remove some high VIF columns
df_2 = df_.drop(columns={"Hip", "Knee", "Wrist", "Chest","Neck","Thigh","Ankle"})

In [ ]:
#Run statsmodesl to get results
X2 = sm.add_constant(df_2)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
#Recheck
calc_vif(df_2)

In [ ]:
#RMSE for smaller model 
x2 = np.array(df_2)
x2.shape
model2 = LinearRegression().fit(x2,y)
ypred2 = model2.predict(x2)
mean_squared_error(y,ypred2,squared=False)

<h3>Smaller Model Results</h3>

Removing variables doesn't up the R2 - we'll explore that more later on in ML.

The RMSE doesn't get much worse (and the change is within the expectation of randomness here) - that makes sense because what we found with this multicollinearity was that each varaible contained the same information. So when we remove one duplicate piece of that information, not much changes. This process is called feature selection, it is one of the big things in ML, and we'll look at it in different ways as we do next semester. 

When we are just using CSVs full of data, there isn't a very large impact in removing a few variables like this. If we were dealing with data that we had to collect - e.g. if we were paying nurses to do all of these measurements on people - then we'd want to minimize the number of features needed, so our nurses could be faster and therefore cheaper. Similarly, if we had massive amounts of data and training our model or storing the data was costly, being able to be accurate with less data is good. We don't often run into constraints in learning, because the data is small, but they can exist. 

In [ ]:
#We can cut it down to the bare-bones - I'll take the vars with small p values
df_3 = df[["Abdomen","Weight"]]
X2 = sm.add_constant(df_3)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
#RMSE for tiny model 
x2 = np.array(df_3)
x2.shape
model2 = LinearRegression().fit(x2,y)
ypred2 = model2.predict(x2)
mean_squared_error(y,ypred2,squared=False)

What's the moral of the story? We can predict almost as well with just two varaibles as we did with all of them!

Challenge - run this with a split of the data. One part to train, another to test. Not the accuracy differences...

In [ ]:
#Run above with test/train split